# Cover Page 
### Student ID: 210003508
### Module Code: GG4257 
### Module Title: Urban Analytics: A Toolkit for Sustainable Urban Development
### Assignment: Lab Assignment No 2 - Networks, Geodemographics and Spatial Microsimulation.
### Degree Programme: Geography 
### Deadline Date: 02.04.2025

In submitting this assignment, I hereby confirm that:

I have read the University's statement on Good Academic Practice; that the following work is my own work; and that significant academic debts and borrowings have been properly acknowledged and referenced.

# Introduction 

This section outlines how to replicate the code and access the required data. All data will be available in a **OneDrive folder**, with the code provided in a dedicated GitHub repository.

This report documents the work conducted for **Lab Assignment 2**, covering challenges from **Labs 5, 6 and 7**. Each lab section includes problem descriptions, methods, and results. Code is supplemented with comments and markdown explanations, with screenshots of outputs (e.g., maps and graphs) included to ensure clarity and help replication. To meet GitHub size limits, certain output cells have been cleared from the notebook. All data paths in the code assume files are stored in a folder named "Data". 

#### GitHub Username: Ejarrettt
#### GitHub Repository: [UA_Lab_Assignment_2](https://github.com/ejarrettt/UA_Lab_Assignment_2)
#### [OneDrive Folder](https://1drv.ms/f/c/46775ff05561cd60/EtpKCVLnUXtMupoCXG5G86QB-GfpojgYMR0f4hgJI-Fclg?e=bvGWOb)

To replicate this report:
1. Clone the GitHub repository.
2. Download the datasets from the [OneDrive Folder](https://1drv.ms/f/c/46775ff05561cd60/EtpKCVLnUXtMupoCXG5G86QB-GfpojgYMR0f4hgJI-Fclg?e=bvGWOb).
3. Unzip the folder and copy the "Data" folder to the same place you have stored the notebook. 
4. Run this notebook. 

# Lab No 5: Introduction to Networks (2 Challenges)

In [ ]:
# Install dependencies for all the Labs 
# ! pip install -r requirements.txt

In [ ]:
import warnings
warnings.simplefilter('ignore', DeprecationWarning)
warnings.filterwarnings('ignore')

In [ ]:
# Import necessary libraries for Lab 5
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import nxviz as nv
import osmnx as ox
import geopandas as gpd

## Challenge 1:

It's time for you to apply everything you learned by analyzing a case study of FourSquare social Network. (Foursquare is a location-based online social network. The dataset contains a list of all of the user-to-user links)

Datasource: @inproceedings{gao2012exploring,
     title={Exploring social-historical ties on location-based social networks},
     author={Gao, Huiji and Tang, Jiliang and Liu, Huan},
     booktitle={Proceedings of the 6th International AAAI Conference on Weblogs and Social Media},
     year={2012}
}

- **Data**: `FS.csv` (avaliable in Moodle)

### Q1. Read the FS network dataset.

In [ ]:
# Load the Foursquare social network dataset into Pandas DataFrame
fs_network = pd.read_csv("/Users/elenajarrett/Library/CloudStorage/OneDrive-Personal/Year 4/GG4257/Data2/Data/data5/FS.csv")

# Print column names to check data structure
print(fs_network.columns)

In [ ]:
# Create a NetworkX graph from the dataset
G = nx.from_pandas_edgelist(fs_network, source='source', target='target')

### Q2. Describe using the basic functions of the graph's size. Explore nodes and edges. Provide how many nodes and edges are present in the network.

In [ ]:
# Display basic properties of the network
print(f"The size of the graph is: {len(G)}") # Total number of nodes
print(f"The number of edges in the graph is: {len(G.edges())}") # Number of connections
print(f"The number of nodes in the graph is: {len(G.nodes())}") # Number of users 

In [ ]:
# Explore graph properties
print(type(G.nodes()))  # Check the type of the nodes list
print(list(G.edges(data=True))[-1])  # Display attributes of the last edge
print(list(G.nodes(data=True))[0])  # Display attributes of the first node
print(type(list(G.edges(data=True))[-1][2]))  # Confirm attribute data type

### Q3. The dataset generates a graph of 639.014 nodes, so it is massive and you won't see anything meaningful if you try to plot it. So you need to create a subset using the **degree centrality** to find out find the top 4 of the most important nodes, and use them to create a subset of the original network.

In [ ]:
nx.number_of_selfloops(G)
# There are no self-loops 

In [ ]:
# Compute degree centrality for all nodes
degree_centrality = nx.degree_centrality(G)

# Get the top 4 nodes with highest degree centrality
top_4_nodes = sorted(degree_centrality, key=degree_centrality.get, reverse=True)[:4]
print("Top 4 nodes with highest degree centrality:", top_4_nodes)

In [ ]:
edges_from_G=G.edges([106223, 89302, 76517, 66999], data=True) #Give me the edges where only 106223 etc. are nodes.
edges_from_G

In [ ]:
G_sub = nx.DiGraph()
len(G_sub)

In [ ]:
G_sub.add_edges_from(edges_from_G) #Adding the list from the subset of nodes.
len(G_sub)

In [ ]:
# Extract edges involving the top 4 nodes
edges_from_G = G.edges(top_4_nodes)

# Create a directed subgraph for the most important nodes
G_sub = nx.DiGraph()
G_sub.add_edges_from(edges_from_G)

In [ ]:
# Create a subgraph of top 4 nodes and their neighbours
nbunch = set(top_4_nodes)  # Start with top 4 nodes
for node in top_4_nodes:
    nbunch.update(G.neighbors(node))  # Add their neighbours

In [ ]:
# Create the subgraph
subG = G.subgraph(nbunch).copy()  # Copy to prevent changes affecting the original graph

In [ ]:
# Print basic info
print(f"Subgraph has {subG.number_of_nodes()} nodes and {subG.number_of_edges()} edges.")

In [ ]:
# Create a subgraph with ONLY the top 4 most important nodes
subG_top4 = G.subgraph(top_4_nodes).copy()

### Q4. Extract the degree centrality values and convert them into a list. Then, plot a histogram to visualize the distribution of node degrees in the original network.

In [ ]:
# Extract degree centrality values and convert them into a list
centrality_list = list(degree_centrality.values())

# Print the first 10 values for reference
print(centrality_list[:10])

In [ ]:
# Create a histogram to visualize the distribution of degree centrality
plt.hist(centrality_list, bins=10, edgecolor="black")
plt.xlabel("Degree Centrality")
plt.ylabel("Number of Nodes")
plt.title("Distribution of Node Degrees")
plt.show()

### Q5. Create a plot for the subset created.

In [ ]:
# Visualise and plot the top 4 nodes subgraph
plt.figure(figsize=(6, 6))
pos = nx.spring_layout(subG_top4, seed=42) 
nx.draw(subG_top4, pos, with_labels=True, node_color='skyblue', edge_color='gray', node_size=500, font_size=8)
plt.title("Subgraph of Only the Top 4 Nodes")
plt.show()

In [ ]:
# Plot the full subgraph (Top 4 nodes + neighbours)
plt.figure(figsize=(8, 8))
nx.draw(G_sub, with_labels=True)
plt.show()

### Q6. Now calculate another relevant measure of the network -- **betweenness centrality**. Plot the betweenness centrality distribution of the subset you created. Tip: Same steps from the previous step, but use `nx.betweenness_centrality()`

In [ ]:
# Create an empty graph to store the network subset
G_network_subset = nx.Graph()
# Add edges from the extracted subgraph to the new graph
G_network_subset.add_edges_from(subG_top4.edges())

# Compute betweenness centrality for all nodes in the subset graph
betweenness_centrality = nx.betweenness_centrality(G_network_subset)

# Convert the betweenness centrality values into a list for analysis
betweenness_list = list(betweenness_centrality.values())

# Print the degree centrality list
#print(betweenness_list) # This outcome is quite long, as a note. 

In [ ]:
# Plot a histogram to visualize the distribution of betweenness centrality
plt.hist(betweenness_list, bins=5) 

# Set axis labels and title
plt.xlabel("Betweenness Centrality")
plt.ylabel("Number of Nodes")
plt.title("Distribution of Betweenness Centrality")

# Plot the histogram 
plt.show()

In [ ]:
# Create an empty graph for visualization
git_network_subset = nx.Graph()

# Add edges from the subset graph correctly
git_network_subset.add_edges_from(subG_top4.edges())  # Extract edges before adding

# Compute betweenness centrality for the visualization graph
betweenness_centrality = nx.betweenness_centrality(git_network_subset)

# Generate node positions for better visualization
pos = nx.spring_layout(git_network_subset, seed=42)

# Draw the graph with node labels and customized appearance
nx.draw(
    git_network_subset, 
    pos, 
    with_labels=True, 
    node_size=700,  # Set node size for better visibility
    node_color='skyblue'  # Use a distinct color for contrast
)

# Add a title to the graph visualization
plt.title("Graph Representation of Subset with Betweenness Centrality")

# Display the graph
plt.show()

### Q7. Plot the Matrix, Arc and Circos from the subset.

In [ ]:
# Adjacency Matrix Plot
nv.MatrixPlot(subG_top4)
plt.title("Adjacency Matrix of Subgraph")
plt.show()

In [ ]:
# Arc Diagram
nv.ArcPlot(subG_top4)
plt.title("Arc Diagram of Subgraph")
plt.show()

In [ ]:
# Circos Plot
nv.CircosPlot(subG_top4)
plt.title("Circos Plot of Subgraph")
plt.show()

## Challenge 2: 

This challenge is about OSMnx. You will explore and analyze a city's street network using the OSMnx Python library.

### Q1. Use OSMnx to download the street network of a city of your choice. You can specify the city name, BBox or a Dict.

First, I download the street network of Birmngham, focusing on New Street Station as the central point. The OSMnx library is then used to retrieve the drivable street network of Birmingham within a 2-mile radius of the central point. The function `graph_from_point()` is used with a "drive" filter to extract only the roads meant for vehicular traffic. After retrieving the network, we visualize it using `ox.plot_graph()`, with no node markers for better readability.

In [ ]:
# Define the central point of Birmingham (New Street Station) and a 2-mile search radius
birmingham_centre = (52.4778, -1.8984)
two_miles = 3218.68  # Distance in meters 

# Retrieve the street network for drivable roads within the defined radius using Bounding Box method 
G_birmingham = ox.graph_from_point(birmingham_centre, dist=two_miles, network_type="drive")

# Plot and visualise the street network
fig, ax = ox.plot_graph(G_birmingham, node_size=0, figsize=(10, 10))
plt.show()

In [ ]:
# Conversion the street network graph to a GDF (nodes and edges)
gdf_nodes, gdf_edges = ox.graph_to_gdfs(G_birmingham)
# Now, `gdf_nodes` contains information about intersections and endpoints,
# and `gdf_edges` contains information about streets and their attributes.

### Q2. Calculate basic statistics for the street network, such as the number of nodes, edges, average node degree, etc.

To analyse the network, we project the graph, compute the area and calculate statitstics. 

In [ ]:
# Finding the area in meters of the map 
warnings.filterwarnings('ignore')

# Project the graph to a coordinate reference system (CRS) with metric units instead of degrees
G_birmingham_proj = ox.project_graph(G_birmingham)

# Extract the nodes from the projected graph as a GeoDataFrame
nodes_proj = ox.graph_to_gdfs(G_birmingham_proj, edges=False)

# Calculate the convex hull area of the network in square meters
graph_area_m = nodes_proj.unary_union.convex_hull.area

# Print the result 
print(f"Graph Area (m²): {graph_area_m}")

In [ ]:
# Calculate basic statistics of the map such as number of nodes, edges, intersection density, etc.
basic_stats = ox.basic_stats(G_birmingham_proj, area=graph_area_m, clean_int_tol=15)
print("Basic Statistics of the Street Network:")
basic_stats

In [ ]:
# Use .edges/.nodes and len() to explicity print the number of nodes and edges 
num_edges = len(list(G_birmingham.edges))
num_nodes = len(list(G_birmingham.nodes))

print(f"Number of edges: {num_edges}")
print(f"Number of nodes: {num_nodes}")

### Q3. Use OSMnx to plot the street network. Customize the plot to make it visually appealing, including node size, edge color. See the potential options here: https://osmnx.readthedocs.io/en/stable/user-reference.html#module-osmnx.plot

In [ ]:
# Customising and plotting the street network of Birmingham using matplotlib
fig, ax = ox.plot_graph(G_birmingham,
                         figsize=(10, 10),  
                         node_size=4,  
                         edge_color="red",  
                         edge_linewidth=0.5,  
                         show=False)  

# Add a title 
plt.title("Street Network around Birmingham New Street Station Within Two Miles") 

# Display the plot 
plt.show()  

I have also created another plot with different customisation. Edges are colored based on their length using the "plasma" colormap. Nodes are colored based on their degree centrality (number of connected streets) using the "viridis" colormap. The background is set to black for improved contrast.

In [ ]:
# Generate edge colors based on their length attribute
edge_colors = ox.plot.get_edge_colors_by_attr(G_birmingham, attr="length", cmap="plasma", start=0, stop=1)

# Generate node colors based on degree centrality (connectivity)
node_colors = ox.plot.get_node_colors_by_attr(G_birmingham, attr="street_count", cmap="viridis", start=0, stop=1)

# Customise and plot the street network with node and edge attributes using osmnx.plot
fig, ax = ox.plot.plot_graph(
    G_birmingham,
    figsize=(10, 10),  
    node_size=10,  
    node_color=node_colors,  # Color nodes based on connectivity
    edge_color=edge_colors,  # Color edges based on length
    edge_linewidth=0.8, 
    bgcolor="black",  
    show=False,
)

# Add a title with styling
plt.title("Street Network around Birmingham New Street Station (2-Mile Radius)", fontsize=14, color="white", pad=20)

# Display the plot
plt.show()

### Q4. Utilize the routing capabilities of OSMnx to find the shortest path between two points in the street network. Plot the route on top of the street network.

The routing process begins by estimating road speeds using `ox.speed.add_edge_speeds(G_birmingham)` and calculating travel times for each road segment with `ox.speed.add_edge_travel_times(G_birmingham)`. The nearest network nodes to Birmingham New Street Station and Aston University are identified using `ox.distance.nearest_nodes()`, and the shortest path, minimizing travel time, is computed with `ox.shortest_path()`. The route is visualized using `ox.plot_graph_route()`, highlighted in yellow with thicker lines for clarity. Distance calculations include the total route length (sum of edge lengths along the path) and the great-circle (haversine) distance, which measures the straight-line distance between the two points. Comparing these distances helps assess the efficiency of the road network by highlighting deviations between direct and real-world travel routes.

In [ ]:
# Add estimated speeds and travel times to edges
G_birmingham = ox.speed.add_edge_speeds(G_birmingham)
G_birmingham = ox.speed.add_edge_travel_times(G_birmingham)

In [ ]:
# Get the nearest street network nodes to two selected lat/long points (origin and destination) using the distance module
orig = ox.distance.nearest_nodes(G_birmingham, X=-1.8984, Y=52.4778)  # Birmingham New Street Station
dest = ox.distance.nearest_nodes(G_birmingham, X=-1.8881, Y=52.4862)  # Aston University

# Print the result 
print(f"Origin Node: {orig}, Destination Node: {dest}")

In [ ]:
# Calculate the shortest route based on travel time 
route = ox.shortest_path(G_birmingham, orig, dest, weight="travel_time")

# Plot the shortest route on top of the street network 
fig, ax = ox.plot_graph_route(G_birmingham, route, node_size=0, route_color="yellow")

In [ ]:
# Calculate the total route distance in meters 
# Extract edge lengths along the route and sum them
edge_lengths = ox.utils_graph.route_to_gdf(G_birmingham, route)["length"]
route_distance_m = round(sum(edge_lengths))

# Print the results
print(f"Total route distance (meters): {route_distance_m}")

In [ ]:
# Calculate the great-circle distance (haversine) between origin and destination 
# # Get the coordinates of the origin and destination nodes
orig_x = G_birmingham.nodes[orig]["x"]
orig_y = G_birmingham.nodes[orig]["y"]
dest_x = G_birmingham.nodes[dest]["x"]
dest_y = G_birmingham.nodes[dest]["y"]

# Calculate great-circle distance (haversine)
haversine_distance_m = round(ox.distance.great_circle(orig_y, orig_x, dest_y, dest_x))

# Print the results
print(f"Great-circle (haversine) distance (meters) between Birmingham New Street Station and Aston University: {haversine_distance_m}")

COULD DO ELEVATION???? 

### Q5. Calculate the centrality measures (e.g., degree centrality and betweenness_centrality) for nodes in the street network.

Centrality measures help assess the importance of nodes and edges in a network. In this analysis, we compute **degree centrality, betweenness centrality, and closeness centrality** for both edges and nodes in Birmingham’s street network. **Closeness centrality**, which indicates how easily a node or edge can access the entire network, is computed by converting the street network to a **line graph** (where edges become nodes) and visualizing the results using a color-coded plot. **Betweenness centrality**, which measures how often an edge appears on the shortest paths between node pairs, is also computed through a **line graph transformation** and visualized based on its values. Similarly, **degree centrality**, which represents the number of direct connections a node (or edge) has, is calculated and displayed using a different colormap. For node-level centrality, we directly compute and store **degree, betweenness, and closeness centrality** as node attributes in the original graph. To effectively visualize these measures, we use `ox.plot_graph()` with different colormaps: **Inferno for betweenness centrality**, highlighting frequently used nodes, **Plasma for degree centrality**, identifying highly connected nodes, and **Viridis for closeness centrality**, showing nodes with the shortest average path to others. This analysis helps in identifying **the most connected, frequently used, and strategically positioned streets** in Birmingham’s road network.

REMOVE AS NOT WHAT QUESTION IS ASKING! 

First, I calculate closeness centrality on Edges. 

In [ ]:
# Convert the graph to a line graph (edges become nodes and vice versa)
edge_centrality = nx.closeness_centrality(nx.line_graph(G_birmingham))

# Store the centrality values as edge attributes in the original graph
nx.set_edge_attributes(G_birmingham, edge_centrality, "edge_centrality")

In [ ]:
# Generate edge colors based on closeness centrality values
ec = ox.plot.get_edge_colors_by_attr(G_birmingham, "edge_centrality", cmap="inferno")

# Plot the graph with edges colored based on closeness centrality
fig, ax = ox.plot_graph(G_birmingham, edge_color=ec, edge_linewidth=2, node_size=0, show=False)

# Add a title 
ax.set_title("Closeness Centrality of Birmingham Street Network (Edges)", fontsize=14, pad=20, color="black")

# Display the plot
plt.show()

Next, I compute Betweenness Centrality on Edges. 

In [ ]:
# Calculate betweenness centrality for edges using a line graph transformation and store in a dictionary
betweenness_centrality = nx.betweenness_centrality(G_birmingham)

# Extracting betweenness_centrality values as a list
betweenness_list = list(betweenness_centrality.values())

# Print the betweeness centrality list (this is not ran as it is very long)
# print(betweenness_list) 

In [ ]:
# Convert the graph to line graph so edges become nodes and vice versa
betweenness_centrality = nx.betweenness_centrality(nx.line_graph(G_birmingham))

# Store the centrality values as edge attributes
nx.set_edge_attributes(G_birmingham, betweenness_centrality, "betweenness_centrality")

# Generate edge colors based on betweenness centrality
ec = ox.plot.get_edge_colors_by_attr(G_birmingham, "betweenness_centrality", cmap="inferno")

# Plot the graph with edges colored based on betweenness centrality
fig, ax = ox.plot_graph(G_birmingham, edge_color=ec, edge_linewidth=2, node_size=0, show=False)

# Add a title 
ax.set_title("Betweenness Centrality of Birmingham Street Network (Edges)", fontsize=14, pad=20, color="black")

# Display the plot
plt.show()

Finally, I calculate the Degree Centrality for Edges. 

In [ ]:
# Calculate degree centrality for edges using a line graph transformation and store in a dictionary
degree_centrality = nx.degree_centrality(G_birmingham)

# Extracting degree centrality values as a list
centrality_list = list(degree_centrality.values())

# Print the degree centrality list (this is not ran as it is very long)
# print(centrality_list) 

In [ ]:
# Convert the graph to line graph so edges become nodes and vice versa
degree_centrality = nx.degree_centrality(nx.line_graph(G_birmingham))

# Store the centrality values as edge attributes
nx.set_edge_attributes(G_birmingham, degree_centrality, "degree_centrality")

# Colour the edges in original graph with degree centralities from line graph
ec = ox.plot.get_edge_colors_by_attr(G_birmingham, "degree_centrality", cmap="inferno")

# Plot the graph with edges colored based on betweenness centrality
fig, ax = ox.plot_graph(G_birmingham, edge_color=ec, edge_linewidth=2, node_size=0, show=False)

# Add a title 
ax.set_title("Degree Centrality of Birmingham Street Network (Edges)", fontsize=14, pad=20, color="black")

# Display the plot
plt.show()

#### Closeness Centrality for Nodes

In [ ]:
# Calculate closeness centrality for each node in the graph
node_closeness_centrality = nx.closeness_centrality(G_birmingham)

# Store the computed centrality values as node attributes in the graph
nx.set_node_attributes(G_birmingham, node_closeness_centrality, "node_closeness_centrality")

In [ ]:
# Generate node colors based on their closeness centrality values 
nc = ox.plot.get_node_colors_by_attr(G_birmingham, "node_closeness_centrality", cmap="inferno")

# Plot the graph with nodes colored based on their closeness centrality
fig, ax = ox.plot_graph(G_birmingham, node_color=nc, node_size=20, edge_linewidth=0.5, show=False)

# Add a title 
ax.set_title("Closeness Centrality of Birmingham Street Network (Nodes)", fontsize=14, pad=20, color="black")

# Display the plot
plt.show()

#### Betweenness Centrality for Nodes

In [ ]:
# Calculate betweenness centrality for each node in the graph
node_betweenness_centrality = nx.betweenness_centrality(G_birmingham)

# Store the computed centrality values as node attributes in the graph
nx.set_node_attributes(G_birmingham, node_betweenness_centrality, "node_betweenness_centrality")


In [ ]:
# Generate node colors based on their betweenness centrality values 
nc = ox.plot.get_node_colors_by_attr(G_birmingham, "node_betweenness_centrality", cmap="inferno")

# Plot the graph with nodes colored based on their betweenness centrality
fig, ax = ox.plot_graph(G_birmingham, node_color=nc, node_size=20, edge_linewidth=0.5, show=False)

# Add a title 
ax.set_title("Betweenness Centrality of Birmingham Street Network (Nodes)", fontsize=14, pad=20, color="black")

# Display the plot
plt.show()

#### Degree Centrality for Nodes

In [ ]:
# Calculate degree centrality for each node in the graph
nodes_degree_centrality = nx.degree_centrality(G_birmingham)

# Store the computed centrality values as node attributes in the graph
nx.set_node_attributes(G_birmingham, nodes_degree_centrality, "nodes_degree_centrality")

In [ ]:
# Generate node colors based on their degree centrality values using the "plasma" colormap
nc = ox.plot.get_node_colors_by_attr(G_birmingham, "nodes_degree_centrality", cmap="plasma")

# Plot the graph with nodes colored based on their degree centrality
fig, ax = ox.plot_graph(G_birmingham, node_color=nc, node_size=20, edge_linewidth=0.5, show=False)

# Add a title 
ax.set_title("Degree Centrality of Birmingham Street Network (Nodes)", fontsize=14, pad=20, color="black")

# Display the plot
plt.show()

### Q6. Create the figure-ground from the selected city

A figure-ground diagram is a visual representation of the urban structure, highlighting streets while leaving the background as empty space. This technique helps to analyse the connectivity and density of an urban area.

In this task, I generated figure-ground diagrams for two locations in Birmingham (New Street Station and the Jewellery Quarter). 

I use OSMnx's plot_figure_ground() function to extract street networks for both areas, customize the figure aesthetics, and save the outputs as images.

In [ ]:
from IPython.display import Image

# Configure image display settings 
img_folder = "images"
extension = "png"
size = 300
dpi = 40

#### Figure-ground for New Street Station in Birmingham

In [ ]:
# Define the location (Birmingham New Street Station)
place = "Birmingham New Street Station"
point = (52.4778, -1.8984) # Lat and Long coordinates 

# Define street widths based on road types
street_widths = {
    "footway": 0.5,
    "steps": 0.5,
    "pedestrian": 0.5,
    "path": 0.5,
    "track": 0.5,
    "service": 2,
    "residential": 3,
    "primary": 5,
    "motorway": 6,
}

# Define file path for saving the image
fp = f"./{img_folder}/{place}.{extension}"

# Create a Matplotlib figure and axis
fig, ax = plt.subplots(figsize=(10, 10))

# Set a black background for better contrast
fig.patch.set_facecolor("black")
ax.set_facecolor("black")

# Create the figure-ground diagram 
ox.plot_figure_ground(
    point=point,
    network_type="all",
    street_widths=street_widths,
    ax=ax,  # Use the predefined Matplotlib axis
    save=False,  # Prevent automatic saving
    show=False,  # Prevent automatic display
)

# Add a title to the plot
ax.set_title(
    f"Figure-Ground Diagram: {place}", 
    fontsize=16, 
    fontweight="bold", 
    color="white",  
    pad=15
)

# Remove axis labels for a clean visualisation
ax.set_xticks([])
ax.set_yticks([])
ax.set_frame_on(False)

# Save the image
plt.savefig(fp, dpi=dpi, bbox_inches="tight", facecolor="black")

# Close the plot to prevent redundant output in Jupyter
plt.close(fig)

# Display the saved image
Image(fp, height=size, width=size)

#### Figure-ground for the Jewellery Quarter in Birmingham 

In [ ]:
# The code is the same as above but the location has changed 
# Define new area in Birmingham - Jewellery Quarter
place = "Birmingham Jewellery Quarter"
point = (52.4862, -1.9134)  

# Define street widths based on road types
street_widths = {
    "footway": 0.5,
    "steps": 0.5,
    "pedestrian": 0.5,
    "path": 0.5,
    "track": 0.5,
    "service": 2,
    "residential": 3,
    "primary": 5,
    "motorway": 6,
}

# Define file path for saving the image
fp = f"./{img_folder}/{place}.{extension}"

# Create a Matplotlib figure and axis
fig, ax = plt.subplots(figsize=(10, 10))

# Set a black background for better contrast
fig.patch.set_facecolor("black")
ax.set_facecolor("black")

# Create the figure-ground diagram 
ox.plot_figure_ground(
    point=point,
    network_type="all",
    street_widths=street_widths,
    ax=ax,  
    save=False,  
    show=False, 
)

# Add a title to the plot
ax.set_title(
    f"Figure-Ground Diagram: {place}", 
    fontsize=16, 
    fontweight="bold", 
    color="white",  # Title color set to white
    pad=15
)

# Remove axis labels for a clean visualisation
ax.set_xticks([])
ax.set_yticks([])
ax.set_frame_on(False)

# Save the image
plt.savefig(fp, dpi=dpi, bbox_inches="tight", facecolor="black")

# Close the plot to prevent redundant output in Jupyter
plt.close(fig)

# Display the saved image
Image(fp, height=size, width=size)

### Q7. Create interactive maps to plot nodes, edges, nodes and edges and one of the centrality measures.

Interactive maps provide a dynamic way to explore Birmingham’s street network. In this task, we first visualize nodes and edges separately to examine their individual structures. Next, we overlay nodes and edges in a single interactive map for a comprehensive view of the network. To enhance understanding of road distribution, edges are colored based on their length, highlighting variations in street segments. Additionally, nodes are color-coded according to betweenness centrality, allowing us to identify key intersections that play a crucial role in connectivity. These visualizations are generated using the folium-based `.explore()` function, which enables interactive exploration with different basemap styles.

#### Interactive Map of Graph Nodes

In [ ]:
# Convert the graph to a GeoDataFrame of nodes
nodes = ox.graph_to_gdfs(G_birmingham, edges=False)

# Create an interactive map of nodes using the "CartoDB Positron" basemap
nodes.explore(tiles="cartodbpositron", marker_kwds={"radius": 8})

#### Interactive Map of Graph Edges 

In [ ]:
# Convert the graph to a GeoDataFrame of edges and explore them interactively
ox.graph_to_gdfs(G_birmingham, nodes=False).explore()

In [ ]:
# Explore edges interactively, colored by their length
edges.explore(tiles="cartodbdarkmatter", column="length", cmap="plasma")

#### Interactive Map of Nodes and Edges Combined 

In [ ]:
# Convert the graph into separate GeoDataFrames for nodes and edges
nodes, edges = ox.graph_to_gdfs(G_birmingham)

# Create an interactive map of edges first
m = edges.explore(color="skyblue", tiles="cartodbdarkmatter")

# Overlay nodes on the same map in yellow
nodes.explore(m=m, color="yellow", marker_kwds={"radius": 3})

#### Interactive Map of Nodes Colored by Betweenness Centrality

In [ ]:
# Compute betweenness centrality for nodes (weighted by road length)
nx.set_node_attributes(G_birmingham, nx.betweenness_centrality(G_birmingham, weight="length"), name="bc")

# Convert the graph to a GeoDataFrame of nodes with the centrality attribute
nodes = ox.graph_to_gdfs(G_birmingham, edges=False)

# Create an interactive map of nodes colored by betweenness centrality
nodes.explore(tiles="cartodbpositron", column="bc", marker_kwds={"radius": 4})

### Q8. Export the street network to a GeoPackage (.gpkg) file. Ensure that the exported file contains both node and edge attributes. Demonstrate that the new GeoPackage can be used and read in Python using any of the libraries we have seen in the class to create a simple and interactive map.

A **GeoPackage (.gpkg)** is a widely used GIS format that stores spatial data in a single, portable file. In this task, we first export the **Birmingham street network** as a GeoPackage, ensuring that both node and edge attributes are preserved. Next, we read the exported file back into Python using **GeoPandas**, allowing us to verify and manipulate the data within a Python environment. Finally, we visualize the network interactively by mapping both **nodes and edges** on a single interactive map, enabling dynamic exploration of the street network. This process ensures that the exported network remains compatible with **GIS software** such as **QGIS and ArcGIS**, while also being seamlessly integrated into Python workflows for further spatial analysis.

In [ ]:
# Export the street network to a GeoPackage file for use in GIS and Python
ox.save_graph_geopackage(G_birmingham, filepath="data/birminghamnetwork.gpkg")


In [ ]:
# Read the GeoPackage file back into Python as GeoDataFrames
gdf_nodes, gdf_edges = gpd.read_file("data/birminghamnetwork.gpkg", layer="nodes"), gpd.read_file("data/birminghamnetwork.gpkg", layer="edges")

In [ ]:
# Create an interactive map to demonstrate that the GeoPackage has been successfully read into Python
# Visualise the edges from the GeoPackage
m = gdf_edges.explore(color="red", tiles="cartodbdarkmatter")

# Overlay nodes onto the same interactive map to show full network connectivity
gdf_nodes.explore(m=m, color="skyblue", marker_kwds={"radius": 6})

### Q9. Finally, use OSMnx to extract other urban elements (e.g., buildings, parks) and plot them.

In [ ]:
# Ignore deprecation warnings
warnings.simplefilter('ignore', DeprecationWarning)

# Extract all building footprints within a 2-mile radius of previously defined centre 
building_footprints = ox.geometries_from_point(
    birmingham_centre,
    tags={"building": True},
    dist=two_miles,
)

# Extract road network for driving
G_birmingham = ox.graph_from_point(
    birmingham_centre, 
    dist=two_miles, 
    network_type="drive"
)

# Extract Railway Network (subway, light rail, tram and rail)
G_rail = ox.graph_from_point(
    birmingham_centre, 
    dist=two_miles, 
    network_type="all", 
    custom_filter='["railway"~"subway|light_rail|tram|rail"]'
)

# Extract parks and greenspaces
parks = ox.geometries_from_point(
    birmingham_centre,
    tags={"leisure": "park"},
    dist=two_miles,
)

# Extract cycleways
cycleways = ox.graph_from_point(
    birmingham_centre, 
    dist=two_miles, 
    network_type="all", 
    custom_filter='["highway"~"cycleway"]'
)

# Create a Figure with a black background
fig, ax = plt.subplots(figsize=(12, 10), facecolor="black")
ax.set_facecolor("black") 

# Plot road network in light gray
ox.plot_graph(G_birmingham, ax=ax, node_size=0, edge_color="lightgray", edge_linewidth=0.5, show=False)

# Plot railway network in red
ox.plot_graph(G_rail, ax=ax, node_size=0, edge_color="red", edge_linewidth=1.0, show=False)

# Plot cycleways in blue
ox.plot_graph(cycleways, ax=ax, node_size=0, edge_color="blue", edge_linewidth=1.2, show=False)

# Plot building footprints in yellow
building_footprints.plot(ax=ax, facecolor="yellow", edgecolor="none", alpha=0.8)

# Plot greenspaces in green
parks.plot(ax=ax, facecolor="green", edgecolor="none", alpha=0.6)

# Add Title 
ax.set_title("Birmingham City Centre: Roads, Railways, Cycleways, Parks, and Buildings", fontsize=16, fontweight="bold", color="white", pad=15)

# Show final plot
plt.show()

WHAT ARE THE BLUE DOTS? 

# Lab No 6: Geodemographics (1 Challenge)

In [ ]:
#Import necessary libraries for Lab 6
import pandas as pd
import geopandas as gpd
import os
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from scipy.spatial.distance import cdist, pdist
import plotly.express as px
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import numpy as np

## Challenge 1: Geodemographic Classification

In this challenge, you will replicate the process of creating a geodemographic classification using the k-means clustering algorithm. Please select any city in the UK except London, Liverpool, or Glasgow. The main goal is to generate a meaningful and informative classification that captures the diversity of areas in your dataset using the census data ( For England, you can try to use the 2021 or 2011 census, and for Scotland, you need to use the 2011 census data) 

### Q1. Define the main goal for the geodemographic classification (marketing, retail and service planning). 

The goal of this project is to support retail planning in Birmingham. By classifying neighbourhoods based on demographic and socio-economic variables, businesses can better understand local markets, strategically place stores, and target consumers more effectively. 

### Q2. Look for census data from the selected city for which you would like to generate the geodemographic classification.

I will use 2021 Census Data for England t the Output Area (OA) level, accessed via [NOMIS](https://www.nomisweb.co.uk/sources/census_2021_bulk). Each CSV file contains a set of variables which will be merged into a single dataset. 

In [ ]:
csv_directory = "/Users/elenajarrett/Library/CloudStorage/OneDrive-Personal/Year 4/GG4257/Data2/Data/data6/census_raw_data/"

# Get a list list of all the CSV files in the folder
csv_files = [file for file in os.listdir(csv_directory) if file.endswith(".csv")]

# Add an empty DataFrame to store the merged data
merged_data = pd.DataFrame()

# Loop through each CSV file - read and mege them column wise
for csv_file in csv_files:
    csv_path = os.path.join(csv_directory, csv_file) # We create a consistent path
    df_csv = pd.read_csv(csv_path, low_memory=False) #read each file
    # Concatenate/Merge all columns, there is a pitfall here, you will get a duplicate oa_code from all csv files.
    merged_data = pd.concat([merged_data, df_csv], axis=1)

# Save the merged dataset
merged_data.to_csv("/Users/elenajarrett/Library/CloudStorage/OneDrive-Personal/Year 4/GG4257/Data2/Data/data6/census_raw_data/merged_census_data.csv", index=False)

Additionally, I found the [output area classification shapefile](https://data.cdrc.ac.uk/dataset/output-area-classification-2011#data-and-resources) for the  UK. I also used local planning authority (LPA) boundaries from a previously used shapefile from Lab No 2 to help define Birmingham's boundary. 

CHANGE LAB NO 2 TO SOURCE 

In [ ]:
# Load UK LPAs and filter for Birmingham 
UK_LPA = "/Users/elenajarrett/Library/CloudStorage/OneDrive-Personal/Year 4/GG4257/Data2/Data/data6/LAP_2021/LPA_MAY_2021_UK_BUC_V2.shp" 
gdf_UK_LPA = gpd.read_file(UK_LPA) 
birmingham_lpa_gdf = gdf_UK_LPA[gdf_UK_LPA.LPA21NM == "Birmingham LPA"] 

# Define the output areas in UK and clip to Birmingham boundary 
oa_shapefile = gpd.read_file("/Users/elenajarrett/Library/CloudStorage/OneDrive-Personal/Year 4/GG4257/Data2/Data/data6/2011_OAC.shp")
birmingham_oa = gpd.clip(oa_shapefile, birmingham_lpa_gdf)

# Save the results 
birmingham_oa.to_file("/Users/elenajarrett/Library/CloudStorage/OneDrive-Personal/Year 4/GG4257/Data2/Data/data6/birmingham_oa.shp", driver='ESRI Shapefile') 

In [ ]:
# Check the datasets structure 
birmingham_oa.head()

In [ ]:
# Mege the census data to Birmingham Output Area Shapefile by mating the OA code 
csv_path = "/Users/elenajarrett/Library/CloudStorage/OneDrive-Personal/Year 4/GG4257/Data2/Data/data6/census_raw_data/merged_census_data.csv"
csv_data = pd.read_csv(csv_path, low_memory=False)
merged_data = birmingham_oa.merge(csv_data, left_on='OA_SA', right_on='geography code', how='left')

In [ ]:
# Check the new dataset 
merged_data.head()

### Q3. The census data at the Output Area OA level. Select multiple topics of at least four topics (socio-demographics, economics, health, and so on). Describe your topic selection accordingly based on the goal of your geodemographic classification. For example, if your geodemographics are related to marketing, Economic variables might be the appropriate selection. 

To develop a robust geodemographic classification, we have selected variables from four key domains that collectively capture demographic structure, economic activity, health status, and living conditions. These dimensions are critical in understanding spatial variations in population characteristics and their implications for consumer behavior, service provision, and urban planning.  

##### **Selected Topics and Justification**  
1. **Socio-Demographics**  
   - Variables: Age distribution, household composition, ethnicity, migration patterns  
   - *Justification*: These factors shape community structures, social needs, and cultural diversity, influencing patterns of service demand and market segmentation.  

2. **Economics**  
   - Variables: Employment status, occupation types, income levels  
   - *Justification*: Economic stability and workforce composition determine purchasing power, economic resilience, and access to financial resources, which are crucial for geodemographic analysis.  

3. **Health**  
   - Variables: General health status, disability prevalence  
   - *Justification*: Health indicators reflect the well-being of communities and can influence service provision, healthcare accessibility, and local policy decisions.  

4. **Housing & Living Conditions**  
   - Variables: Housing tenure, overcrowding, access to essential services  
   - *Justification*: Housing stability and living conditions impact quality of life, neighborhood desirability, and long-term socio-economic outcomes.  

##### **Selected 2021 Census Datasets**  
To operationalise these topics, we will utilize the following census datasets:  

- **Population & Demographics**  
  - **TS001**: Number of usual residents in households and communal establishments  
  - **TS006**: Population density  
  - **TS008**: Sex distribution  
  - **TS007A**: Age by five-year age bands  

- **Economic Activity**  
  - **TS058**: Distance travelled to work (mobility patterns)  
  - **TS063**: Occupation types (workforce distribution)  
  - **TS066**: Economic activity status (employment/unemployment levels)  
  - **TS067**: Highest level of qualification (educational attainment)  
  - **TS068**: Schoolchildren and full-time students (education participation)  

- **Health & Well-being**  
  - **TS037**: General health (self-reported health status)  
  - **TS038**: Disability prevalence (long-term illness or disability)  

- **Housing & Living Conditions**  
  - **TS011**: Households by deprivation dimensions (multi-dimensional deprivation indicators)  
  - **TS021**: Ethnic group (diversity & integration)  
  - **TS029**: Proficiency in English (language barriers & social inclusion)  
  - **TS041**: Number of households (housing stock and population distribution)  
  - **TS054**: Tenure (owner-occupied vs. rental market trends)  

##### **Relevance to Geodemographic Classification**  
These datasets provide a multi-faceted understanding of population characteristics, economic conditions, and residential patterns. By integrating these variables, we can classify geographical areas into meaningful segments, helping policymakers, businesses, and urban planners tailor services and interventions to meet the diverse needs of different communities.  

### Q4. Identify the variables that will be crucial for effectively segmenting neighbourhoods. Evaluate how this choice may impact the classification results, including a DEA analysis.

Key variables chosen for clustering:
   - **Socio-demographics**: Percentage of young (0-18), working-age (18-64), and elderly (65+)
   - **Economics**: Unemployment rate, percentage of professionals vs. manual laborers
   - **Health**: Percentage reporting ‘bad’ or ‘very bad’ health
   - **Housing**: Percentage of households in social housing, percentage of owner-occupied homes

Impact Analysis:
   - A high percentage of young people may indicate demand for educational services and youth-oriented retail.
   - High unemployment rates may reflect areas with lower spending power.
   - Poor health indicators could highlight areas needing healthcare services and accessible facilities.
   - Housing tenure affects stability and service demand—areas with more rented accommodations may have higher population turnover.

In [ ]:
list(merged_data.columns)
#This is just a small sample of the number of potential variables you have available in the census
# Using some ML techniques, you could include hundreds of these to make your geodemographics
# more meaningful. For this academic exercise, only 386 variables will be loaded,
# and then filtered to be included in the cluster

In [ ]:
merged_data.info()

In [ ]:
merged_data.shape

In [ ]:
# List of selected variables for summary statistics
selected_variables = [
    'Population Density: Persons per square kilometre; measures: Value',
    'Ethnic group: White',  
    'Economic activity status: Economically active (excluding full-time students): Unemployed',
    'Occupation (current): 2. Professional occupations',
    'General health: Bad health',
    'Disability: Disabled under the Equality Act: Day-to-day activities limited a lot',
    'Household deprivation: Household is deprived in two dimensions; measures: Value',
    'Tenure of household: Private rented',
    'Distance travelled to work: Works mainly from home',
    'Highest level of qualification: Level 4 qualifications and above'
]

# Generate summary statistics for selected variables
merged_data[selected_variables].describe()


#### DEA Analysis - Visualising Variable Distribution

In [ ]:
# Define key DEA-related variables for visualization
attributes_to_plot = [
    'Economic activity status: Economically active (excluding full-time students): Unemployed',
    'Household deprivation: Household is deprived in two dimensions; measures: Value',
    'General health: Bad health',
    'Disability: Disabled under the Equality Act: Day-to-day activities limited a lot',
    'Occupation (current): 2. Professional occupations',
    'Highest level of qualification: Level 4 qualifications and above',
    'Tenure of household: Owned',
    'Distance travelled to work: Works mainly from home',
    'Population Density: Persons per square kilometre; measures: Value'
]

# Create figure with 3x3 grid
fig, axes = plt.subplots(3, 3, figsize=(14, 12))

# Flatten the axes array to easily loop over it
axes = axes.flatten()

# Loop through attributes and assign plots to subplots
for i, attribute in enumerate(attributes_to_plot):
    sns.violinplot(x=merged_data[attribute], color="skyblue", ax=axes[i])
    axes[i].set_title(attribute, fontsize=10, rotation=10)  # Shortened & rotated titles
    axes[i].set_xlabel('')  # Remove x-axis label
    axes[i].set_ylabel('')

# Adjust layout to fully remove title overlap
plt.subplots_adjust(hspace=0.8, wspace=0.5)  # Increased spacing
plt.show()


In [ ]:
# List of attributes to plot (now 9 attributes)
attributes_to_plot = [
    'Economic activity status: Economically active (excluding full-time students): Unemployed',
    'Household deprivation: Household is deprived in two dimensions; measures: Value',
    'General health: Bad health',
    'Disability: Disabled under the Equality Act: Day-to-day activities limited a lot',
    'Occupation (current): 2. Professional occupations',
    'Highest level of qualification: Level 4 qualifications and above',
    'Tenure of household: Owned',
    'Distance travelled to work: Works mainly from home',
    'Population Density: Persons per square kilometre; measures: Value'
]

# Create figure with 3x3 grid
fig, axes = plt.subplots(3, 3, figsize=(14, 12))

# Loop through attributes and assign plots to subplots
for i, attribute in enumerate(attributes_to_plot):
    row, col = divmod(i, 3)  # Determine subplot position
    sns.histplot(merged_data[attribute], kde=True, bins=20, color="skyblue", ax=axes[row, col])
    axes[row, col].set_title(attribute, fontsize=7, wrap=True, rotation=10)  # Fix title overlap
    axes[row, col].set_xlabel('')  # Remove x-axis label
    axes[row, col].set_ylabel('Frequency')

# Adjust layout to fully remove title overlap
plt.subplots_adjust(hspace=0.8, wspace=0.5)  # Increased spacing
plt.show()

In [ ]:
attributes_to_plot = ['Economic activity status: Economically active (excluding full-time students): Unemployed',
    'Household deprivation: Household is deprived in two dimensions; measures: Value',
    'General health: Bad health',
    'Disability: Disabled under the Equality Act: Day-to-day activities limited a lot',]

plt.figure(figsize=(12, 8))

for i, attribute in enumerate(attributes_to_plot, 1):
    plt.subplot(2, 2, i)
    sns.histplot(merged_data[attribute].astype(str), kde=True)
    plt.title(attribute)

plt.tight_layout()
plt.show()

### Q5. Prepare, adjust or clean the dataset addressing any missing values or outliers that could distort the clustering results.

**1. Socio-Demographics (7 variables)**
- `Population Density: Persons per square kilometre; measures: Value` → **Indicates potential footfall for retail locations.**  
- `Age: Aged 20 to 24 years` → **Captures younger consumers, important for fashion, entertainment, and fast food.**  
- `Age: Aged 25 to 29 years` → **Captures working professionals with disposable income.**  
- `Age: Aged 30 to 34 years` → **Represents key consumers for home goods, groceries, and family-related purchases.**  
- `Age: Aged 65 years and over` → **Captures older consumers who may need accessible shopping options.**  
- `Ethnic group: Total: All usual residents` → **Demographic diversity influences demand for specific products.**  
- `Proficiency in English language: Main language is not English (English or Welsh in Wales): Cannot speak English well` → **Retailers may need multilingual signage, marketing, or staff.**  


**2. Economic Activity & Occupation (8 variables)**
- `Economic activity status: Economically active (excluding full-time students): In employment` → **Indicates the proportion of working residents who have disposable income.**  
- `Economic activity status: Economically active (excluding full-time students): Unemployed` → **High unemployment may indicate lower retail spending.**  
- `Economic activity status: Economically inactive: Retired` → **Captures areas with older consumers who may have different shopping habits.**  
- `Occupation (current): 2. Professional occupations` → **Higher share of professionals indicates greater spending capacity.**  
- `Occupation (current): 7. Sales and customer service occupations` → **Workers in these roles are key for retail employment and consumer trends.**  
- `Occupation (current): 9. Elementary occupations` → **Lower-income workers may require budget-friendly retail options.**  
- `Highest level of qualification: Level 4 qualifications and above` → **Higher education levels correlate with higher spending power.**  
- `Distance travelled to work: Less than 2km` → **Indicates local workforce that supports nearby retail businesses.**  

**3. Health Indicators (3 variables)**
- `General health: Very good health` → **Healthier populations are likely to engage in leisure shopping and social activities.**  
- `General health: Bad health` → **Higher rates of poor health may impact consumer mobility and retail accessibility needs.**  
- `Disability: Disabled under the Equality Act: Day-to-day activities limited a lot` → **Indicates demand for accessible retail stores and services.**  

**4. Housing & Living Conditions (7 variables)**
- `Tenure of household: Owned` → **Homeownership often correlates with stable communities and higher retail spending.**  
- `Tenure of household: Private rented` → **Higher rental areas may have more transient populations with different retail needs.**  
- `Tenure of household: Social rented` → **Lower-income areas may require affordable retail options.**  
- `Household deprivation: Household is deprived in two dimensions; measures: Value` → **Indicates economic disadvantage, influencing retail demand for budget stores.**  
- `Household deprivation: Household is deprived in three dimensions; measures: Value` → **High deprivation areas may have lower retail spending power.**  
- `Number of households: Number of households; measures: Value` → **Total household count helps estimate the size of the retail market.**  
- `Distance travelled to work: Works mainly from home` → **Higher remote work levels can impact demand for local retail services.**  

**Why These 25 Variables?**
- **Consumer Demographics** → Age, ethnicity, language proficiency influence product demand.  
- **Spending Power & Employment** → Economic activity, occupation, and education level impact disposable income.  
- **Retail Accessibility & Footfall** → Population density, homeownership, and workforce commuting patterns affect store viability.  
- **Health & Disability** → Affects shopping habits, accessibility requirements, and demand for health-related retail.  

In [ ]:
def calculate_percentages(dataframe, total_columns, value_columns):
    """
    Calculate percentage values for selected variables based on corresponding totals.

    Parameters:
    - dataframe (pd.DataFrame): The dataset containing the relevant variables.
    - total_columns (list): List of total population columns corresponding to value columns.
    - value_columns (list): List of specific value columns to be converted into percentages.

    Returns:
    - result_df (pd.DataFrame): DataFrame containing the percentage values.
    """
    result_df = pd.DataFrame()

    for total_col, value_col in zip(total_columns, value_columns):
        percentage_col_name = f"{value_col}_percentage"

        if total_col not in dataframe.columns or value_col not in dataframe.columns:
            print(f"Warning: '{total_col}' or '{value_col}' not found in DataFrame. Skipping...")
            continue  # Skips missing columns instead of raising an error

        # Convert columns to numeric or NaN if errors occur
        dataframe[value_col] = pd.to_numeric(dataframe[value_col], errors='coerce')
        dataframe[total_col] = pd.to_numeric(dataframe[total_col], errors='coerce')
        
        result_df[percentage_col_name] = (dataframe[value_col] / dataframe[total_col]) * 100

    return result_df

# Updated total columns (Matching Dataset)
total_cols = [
    'Age: Total',
    'Age: Total',
    'Age: Total',
    'Age: Total',
    'Ethnic group: Total: All usual residents',
    'Sex: All persons; measures: Value',
    'Sex: All persons; measures: Value',
    'Economic activity status: Total: All usual residents aged 16 years and over',
    'Economic activity status: Total: All usual residents aged 16 years and over',
    'Economic activity status: Total: All usual residents aged 16 years and over',
    'Economic activity status: Total: All usual residents aged 16 years and over',
    'Occupation (current): Total: All usual residents aged 16 years and over in employment the week before the census',
    'Occupation (current): Total: All usual residents aged 16 years and over in employment the week before the census',
    'Occupation (current): Total: All usual residents aged 16 years and over in employment the week before the census',
    'General health: Total: All usual residents',
    'General health: Total: All usual residents',
    'Disability: Total: All usual residents',
    'Disability: Total: All usual residents',
    'Tenure of household: Total: All households',
    'Tenure of household: Total: All households',
    'Household deprivation: Total: All households; measures: Value',
    'Household deprivation: Total: All households; measures: Value',
    'Distance travelled to work: Total: All usual residents aged 16 years and over in employment the week before the census'
]

# Updated value columns (Matching Dataset)
value_cols = [
    'Age: Aged 20 to 24 years',
    'Age: Aged 25 to 29 years',
    'Age: Aged 30 to 34 years',
    'Age: Aged 35 to 39 years',
    'Ethnic group: White',
    'Sex: Female; measures: Value',
    'Sex: Male; measures: Value',
    'Economic activity status: Economically active (excluding full-time students):In employment',
    'Economic activity status: Economically active (excluding full-time students): Unemployed',
    'Economic activity status: Economically inactive: Retired',
    'Economic activity status: Economically inactive: Long-term sick or disabled',
    'Occupation (current): 2. Professional occupations',
    'Occupation (current): 7. Sales and customer service occupations',
    'Occupation (current): 9. Elementary occupations',
    'General health: Very good health',
    'General health: Bad health',
    'Disability: Disabled under the Equality Act: Day-to-day activities limited a lot',
    'Disability: Disabled under the Equality Act: Day-to-day activities limited a little',
    'Tenure of household: Owned',
    'Tenure of household: Private rented',
    'Household deprivation: Household is deprived in two dimensions; measures: Value',
    'Household deprivation: Household is deprived in three dimensions; measures: Value',
    'Distance travelled to work: Works mainly from home'
]

# Apply function to calculate percentages
result_dataframe = calculate_percentages(merged_data, total_cols, value_cols)


In [ ]:
result_dataframe.head()

In [ ]:
result_dataframe.shape

In [ ]:
# Concatenate the resulting tables.
concatenated_df = pd.concat([merged_data, result_dataframe], axis=1, ignore_index=False)
concatenated_df.head()

In [ ]:
concatenated_df.shape

In [ ]:
list(concatenated_df.columns)

In [ ]:
# Subsetting the attributes we need, we dont need the total for now.
keep_cols= [
    'OA_SA',
    'geometry',
    'Population Density: Persons per square kilometre; measures: Value',
    'Age: Aged 20 to 24 years_percentage',
    'Age: Aged 25 to 29 years_percentage',
    'Age: Aged 30 to 34 years_percentage',
    'Age: Aged 35 to 39 years_percentage',
    'Ethnic group: White_percentage',
    'Sex: Female; measures: Value_percentage',
    'Sex: Male; measures: Value_percentage',
    'Economic activity status: Economically active (excluding full-time students):In employment_percentage',
    'Economic activity status: Economically active (excluding full-time students): Unemployed_percentage',
    'Economic activity status: Economically inactive: Retired_percentage',
    'Economic activity status: Economically inactive: Long-term sick or disabled_percentage',
    'Occupation (current): 2. Professional occupations_percentage',
    'Occupation (current): 7. Sales and customer service occupations_percentage',
    'Occupation (current): 9. Elementary occupations_percentage',
    'General health: Very good health_percentage',
    'General health: Bad health_percentage',
    'Disability: Disabled under the Equality Act: Day-to-day activities limited a lot_percentage',
    'Disability: Disabled under the Equality Act: Day-to-day activities limited a little_percentage',
    'Tenure of household: Owned_percentage',
    'Tenure of household: Private rented_percentage',
    'Household deprivation: Household is deprived in two dimensions; measures: Value_percentage',
    'Household deprivation: Household is deprived in three dimensions; measures: Value_percentage',
    'Distance travelled to work: Works mainly from home_percentage'
]

birmingham_census_data = concatenated_df[keep_cols]

In [ ]:
birmingham_census_data.head()

In [ ]:
# For more easy manipulation we define short column names

short_column_names = {
    'OA_SA': 'output_areas',
    'geometry': 'geometry',
    'POPULATION': 'population',
    'Population Density: Persons per square kilometre; measures: Value': 'pop_density',
    'Age: Aged 20 to 24 years_percentage': 'age_20_24',
    'Age: Aged 25 to 29 years_percentage': 'age_25_29',
    'Age: Aged 30 to 34 years_percentage': 'age_30_34',
    'Age: Aged 35 to 39 years_percentage': 'age_35_39',
    'Ethnic group: White_percentage': 'eth_white',
    'Sex: Female; measures: Value_percentage': 'female',
    'Sex: Male; measures: Value_percentage': 'male',
    'Economic activity status: Economically active (excluding full-time students):In employment_percentage': 'emp_active',
    'Economic activity status: Economically active (excluding full-time students): Unemployed_percentage': 'emp_unemp',
    'Economic activity status: Economically inactive: Retired_percentage': 'inactive_retired',
    'Economic activity status: Economically inactive: Long-term sick or disabled_percentage': 'inactive_sick',
    'Occupation (current): 2. Professional occupations_percentage': 'occ_prof',
    'Occupation (current): 7. Sales and customer service occupations_percentage': 'occ_sales',
    'Occupation (current): 9. Elementary occupations_percentage': 'occ_elem',
    'General health: Very good health_percentage': 'health_very_good',
    'General health: Bad health_percentage': 'health_bad',
    'Disability: Disabled under the Equality Act: Day-to-day activities limited a lot_percentage': 'disab_limited_lot',
    'Disability: Disabled under the Equality Act: Day-to-day activities limited a little_percentage': 'disab_limited_little',
    'Tenure of household: Owned_percentage': 'tenure_owned',
    'Tenure of household: Private rented_percentage': 'tenure_rented',
    'Household deprivation: Household is deprived in two dimensions; measures: Value_percentage': 'deprived_2dim',
    'Household deprivation: Household is deprived in three dimensions; measures: Value_percentage': 'deprived_3dim',
    'Distance travelled to work: Works mainly from home_percentage': 'work_home'
}

birmingham_census_data = birmingham_census_data.rename(columns=short_column_names)

In [ ]:
birmingham_census_data.head()

In [ ]:
birmingham_census_data.info()

### Q6. Include standardisation between areas and variables. Make an appropriate analysis and adjust the variable selection accordingly for any multicollinearity.

In [ ]:
# Calculate z-score for each column, but we need to initially filter only the float attributes.
# bcs you can't calculate that for the OA code.. :P

numeric_columns = birmingham_census_data.select_dtypes(include='float64')
z_score_df = (numeric_columns - numeric_columns.mean()) / numeric_columns.std(ddof=0)
z_score_df.head()

In [ ]:
z_score_df.shape

In [ ]:
corr = z_score_df.corr()
corr.style.background_gradient(cmap='coolwarm')

In [ ]:
plt.matshow(z_score_df.corr())
plt.show()

In [ ]:
threshold = 0.7 # We can adapt this based on waht we have in our data. Recall the subjetivity issue?
# So if we include .8 then we wont be able to reduce any variables., so I took the threshold to 70%

highly_correlated = (corr.abs() > threshold) & (corr.abs() < 1.0)

plt.figure(figsize=(10, 8))
sns.heatmap(highly_correlated, cmap='coolwarm', cbar=False, annot=True)

plt.title('Highly Correlated Variables')
plt.show()

# The plot will represent  BINARY table 0 for false( out of the threshold) and 1 for above the threshold.
# but I also coloured so it is easier to use and cute :) 

In [ ]:
z_score_df.info()

In [ ]:
z_score_df.drop(['female', 'work_home', 'disab_limited_lot'], axis=1, inplace=True)
z_score_df.info()

In [ ]:
corr_2 = z_score_df.corr()
corr_2.style.background_gradient(cmap='coolwarm')

In [ ]:
threshold = 0.7
highly_correlated_2 = (corr_2.abs() > threshold) & (corr_2.abs() < 1.0)

plt.figure(figsize=(10, 8))
sns.heatmap(highly_correlated_2, cmap='coolwarm', cbar=False, annot=True)

plt.title('New Highly Correlated Variables')
plt.show()

In [ ]:
contains_nan = z_score_df.isna().any().any()

if contains_nan:
    print("Oh no :( the DataFrame contains NaN values.")
else:
    print("Wahoo! the dataFrame does not contain NaN values. I'm the best")

In [ ]:
z_score_df.fillna(z_score_df.mean(), inplace=True)

In [ ]:
z_score_df.head()

In [ ]:
z_score_df.shape

### Q7. Utilize the k-means clustering algorithm to create a classification based on the selected variables.

We will use the k-means clustering method, the same approach used to produce the Output Area Classifications you can find in multiple website in the UK. It is a top-down approach whereby the number of cluster groups is predefined. K-means is an iterative relocation algorithm based on an error sum of squares measure. The algorithm seeks to reduce the sum distance between each data point and their respective cluster centre. The diagram below illustrates the basic algorithm process of k-means clustering. 

It starts by randomly allocating seeds across a multidimensional space as defined by the variables, each case is then assigned to the nearest seed centroid. In other words, the cases are assigned into cluster groups based on the seed they are nearest to across the multiple variables.

Following the first iteration, a new seed is created at the centroid of each of the clusters. Each case is then re-assigned to clusters based on the distance to the nearest of these new centroids. This process repeats iteratively until the centroid seed locations cannot be moved as an optimum solution has been reached (See Harris et al., 2005).

The process of the k-means algorithm (taken from Lansley et al, 2015). The code is annotated below., but you can get all the parameter by running `?Kmeans`


In [ ]:
#?KMeans

In [ ]:
from sklearn.cluster import KMeans
from scipy.spatial.distance import cdist, pdist

# When I first ran this code, I found that I kept getting different results. This is because, as discussed in class, 
# KMeans is a stocastic method. Therefore, I added a random_state and an arbitrary number so that the maps that you create will be the same. 

# KMeans with 10 clusters
kmeans = KMeans(n_clusters=10)
kmeans.fit(z_score_df)
labels = kmeans.predict(z_score_df)
cluster_centres = kmeans.cluster_centers_
z_score_df['Cluster'] = kmeans.labels_

In [ ]:
z_score_df.head()

In [ ]:
plt.hist(labels)
# Ok, we clustered the data, but we subjectively defined the number of clusters, 
# and as you can see is one of the key parameters

### Q8. Define the optimum number of clusters (i.e., using the Elblow method). Experiment with different values of k.

#### Testing using plots 

In [ ]:
# Let's try doing the same plot as above but with different values 
# KMeans with 8 clusters
kmeans = KMeans(n_clusters=8, random_state=9) # Trying 8
kmeans.fit(z_score_df)
labels = kmeans.predict(z_score_df)
cluster_centres = kmeans.cluster_centers_
z_score_df['Cluster'] = kmeans.labels_
plt.hist(labels)

In [ ]:
kmeans = KMeans(n_clusters=6, random_state=9) # Trying 6
kmeans.fit(z_score_df)
labels = kmeans.predict(z_score_df)
cluster_centres = kmeans.cluster_centers_
z_score_df['Cluster'] = kmeans.labels_
plt.hist(labels)

In [ ]:
kmeans = KMeans(n_clusters=5, random_state=9) # Trying 5
kmeans.fit(z_score_df)
labels = kmeans.predict(z_score_df)
cluster_centres = kmeans.cluster_centers_
z_score_df['Cluster'] = kmeans.labels_
plt.hist(labels)

In [ ]:
kmeans = KMeans(n_clusters=4, random_state=9) # Trying 4
kmeans.fit(z_score_df)
labels = kmeans.predict(z_score_df)
cluster_centres = kmeans.cluster_centers_
z_score_df['Cluster'] = kmeans.labels_
plt.hist(labels)

#### Elbow Method 

In [ ]:
Sum_of_squared_distances = []

K_range = range(1,15)

for k in K_range:
 km = KMeans(n_clusters=k)
 km = km.fit(z_score_df)
 Sum_of_squared_distances.append(km.inertia_)
    
plt.plot(K_range, Sum_of_squared_distances, 'bx-')
plt.xlabel('k')
plt.ylabel('Sum_of_squared_distances')
plt.title('Elbow Method For Optimal k')
plt.show()

The 'elbow' of the arm is around 4-5, which means we should try using a cluster number value close to 4-5. 

#### Between-Cluster Squares 

In [ ]:
def elbow(dataframe, n):
    kMeansVar = [KMeans(n_clusters=k).fit(dataframe.values) for k in range(1, n)] #making use of list comprehensions.
    centroids = [X.cluster_centers_ for X in kMeansVar]
    k_euclid = [cdist(dataframe.values, cent) for cent in centroids]
    dist = [np.min(ke, axis=1) for ke in k_euclid]
    wcss = [sum(d**2) for d in dist]
    tss = sum(pdist(dataframe.values)**2)/dataframe.values.shape[0]
    bss = tss - wcss
    plt.plot(bss)
    plt.show()
 
elbow(z_score_df,15)

In [ ]:
# KMeans with 6 clusters, after the validation with the Elbow method.
kmeans = KMeans(n_clusters=5)
kmeans.fit(z_score_df)
labels = kmeans.predict(z_score_df)
cluster_centres = kmeans.cluster_centers_

z_score_df['Cluster'] = kmeans.labels_

In [ ]:
z_score_df.head()

EXPLORE WITH DIFFERENT k VALUES

**What is the optimum number of clusters?**

There is no right answer to this question. Even making judgements using some guidance on criteria involves a level of subjectivity. Your task is now to choose an appropriate number of clusters for your geodemographic classification.

Aims of the cluster analysis:
* Each cluster should be homogeneous as possible
* Each cluster group should be distinct from the other groups
* The groups should be as evenly sized as possible

In addition, to each of these, we must also consider the compositions of the cluster groups. It is important that each of the characteristics of each cluster are distinguishable and relatable to real-life neighbourhoodtypes. 

Each cluster should be homogeneous as possible. Generally, the greater the number of clusters the closer the each case is to their cluster centroid on average. However, of course, with more groups the classification becomes more difficult to interpret and the differences between some groups may become more subtle. 

A measure we can use to check this is the within-cluster sum of squares which is the sum of the squared deviations from each observation to the cluster centroid. So larger sums indicate that the cluster is more dispersed and less homogenous. A plot of the within groups sum of squares by number of clusters extracted can help determine the appropriate number of clusters (k). The tip is to look for a bend in the plot similar to a scree test in factor analysis (called the elbow method).

Each cluster group should be distinct from the other groups We can also measure how distinctive each of the clusters are in each model overall by looking at the between-cluster sum of squares. This value essentially measures how far apart clusters from different centres are. If the value is low then cases from different clusters will not be that distinctive. It is also important to observe the cluster centres to ensure that the compositions of each of the groups are logical and sufficiently unique.Having selected the number of clusters for the final model, below are two ways of visualising the fit of a k-means model, you may need to install the packages first (as described earlier). 

In the example 5 = k, and the data corresponds with all of the Output Areas. This Creates a bivariate plot visualising a partition (clustering) of the data. All observation are represented by points in the plot, using **principal components or multidimensional scaling**. Around each cluster, an ellipse is drawn.

See here for the full documentation about the implementation of Kmeans and how you can validate your results:

* https://scikit-learn.org/stable/auto_examples/decomposition/plot_pca_iris.html#sphx-glr-auto-examples-decomposition-plot-pca-iris-py
* https://scikit-learn.org/stable/modules/clustering.html#clustering-evaluation
* https://scikit-learn.org/stable/modules/clustering.html#k-means
* https://scikit-learn.org/stable/auto_examples/cluster/plot_kmeans_assumptions.html#sphx-glr-auto-examples-cluster-plot-kmeans-assumptions-py

**Using the two plots, is it possible to determine an appropriate number of groups?**

There are also other methods for identifying statistically optimal groups such as **the average silhouette method** and **gap statistic method**.  

The groups should be as evenly sized as possible. This is more of a rule of thumb for geodemographics. However, if a single cluster group comprises a large share of the data, then it is obviously undesirable for practitioners hoping to glean a better understanding of how the needs of neighbourhoods may vary. 

You might notice that your K-means has produced a small cluster group which has exaggerated cluster centre values for a small number of variables, this is not unusual when attempting to segment social  groups. 

There are two main ways to edit the size of the groups without manipulating the variables: 

* joining similar groups or
* splitting large groups.

If one cluster group is particularly dominant in size, you can force it into two groups by isolating the cases (statistical areas) from that group only and running a 2 group solution k-means. With only one group selected, you can run a k-means as you did before, but change the number of groups produced to 2. As only one group is selected, the output areas from other groups will be excluded from the analysis. If you are happy with the results you now need to combine the two new groups with the original group membership variable.

### Q9. Evaluate your cluster groups (e.g., using PCA) and interpret your cluster centres. Describe your results and repeat the process to adjust the variable selection and cluster groups to provide more meaningful results for your geodemographic goal. Interpret the characteristics of each cluster. What demographic patterns or similarities are prevalent within each group?

MAKE SURE TO ANS QUESTION 

In [ ]:
# Code based on the example provided here: 
# https://scikit-learn.org/stable/auto_examples/decomposition/plot_pca_iris.html#sphx-glr-auto-examples-decomposition-plot-pca-iris-py

plt.figure(figsize=(12, 8))

kmeans = KMeans(n_clusters=5)
clusters = kmeans.fit_predict(z_score_df)

z_score_df['Cluster'] = clusters

scaler = StandardScaler()
stand_data_scaled = scaler.fit_transform(z_score_df)

# PCA analysys.
pca = PCA(n_components=2).fit(stand_data_scaled)
pca_result = pca.transform(stand_data_scaled)

#Percentage of variance explained by each of the selected components.
variance_ratio = pca.explained_variance_ratio_

# Create a scatter plot
fig = px.scatter(x=pca_result[:, 0], y=pca_result[:, 1], color=clusters,
                 labels={'color': 'Cluster'},
                 #title='Cluster Plot against 1st 2 Principal Components',
                 opacity=0.7,
                 width=800, 
                 height=800)

plt.tight_layout()
fig.show()

print(f"These two components explain {(variance_ratio.sum()*100):.2f}% of the point variability.")


In [ ]:
# Here is a static figure with the point variability included in the x/y-axis label.
# So we can see what variability is provided by each component.

kmeans = KMeans(n_clusters=5)
clusters = kmeans.fit_predict(z_score_df)

z_score_df['Cluster'] = clusters

# Standardize the data for PCA
scaler = StandardScaler()
stand_data_scaled = scaler.fit_transform(z_score_df)

# PCA
pca = PCA(n_components=2).fit(stand_data_scaled)
pca_result = pca.transform(stand_data_scaled)

#Percentage of variance explained by each of the selected components.
variance_ratio = pca.explained_variance_ratio_

plt.figure(figsize=(10, 6))
sns.scatterplot(x=pca_result[:, 0], y=pca_result[:, 1], hue=clusters, palette='viridis', s=50, alpha=0.7)
plt.title('Cluster Plot against 1st 2 Principal Components')
plt.xlabel(f'Principal Component 1 variation: {variance_ratio[0]*100:.2f}%')
plt.ylabel(f'Principal Component 2 variation: {variance_ratio[1]*100:.2f}%')
plt.legend(title='Clusters')
plt.show()


#### Interpreting Clusters 

In [ ]:
# KMeans clustering
kmeans = KMeans(n_clusters=5)
clusters = kmeans.fit_predict(z_score_df)

# Get the cluster centers
cluster_centers = kmeans.cluster_centers_


# Get the cluster centers
cluster_centers = pd.DataFrame(kmeans.cluster_centers_, columns=z_score_df.columns)

# Create a new DataFrame with cluster assignments and column names
#result_df = pd.DataFrame({'Cluster': clusters, 'Column': z_score_df.columns})

cluster_centers.head(6)

**Interpreting the cluster centres**

Before any efforts are made to visualise the data, it is important you understand what it represents. The cluster centres (`kmeans.cluster_centers_`), indicates the coordinates of the centroid for each cluster group once the k-means had reached its optimum solution. It, therefore, is a good indicator of the average characteristics of each group based on the n variables that were included in the original model.

We inputted **Z-score standardised data** into the model, therefore the cluster centres are still represented as Z-scores. Zero represents the mean for each variable and values above or below indicate the number of standard deviations away from the average. The values can, therefore, be used to very easily understand how unique each group is relative to the whole sample (in this case, all of the pop_data you inputted).

**What if we print the cluster centrers?** --**Is it immediately clear what your groups represent?**

It might be easier to create charts to visualise the characteristics of each cluster group. In the example below we will create radial plots, using the first group as our example.

In [ ]:
# Step 1: Select the centre values for Cluster 0 (the first cluster)
first_row_centers = cluster_centers.iloc[0, :]

# Step 2: Count the number of features (i.e., variables used in clustering)
num_features = len(first_row_centers)

# Step 3: Generate evenly spaced angles (in radians) for each feature around a circle
theta = np.linspace(0, 2 * np.pi, num_features, endpoint=True)

# Step 4: Create a polar plot using matplotlib
fig, ax = plt.subplots(subplot_kw={'projection': 'polar'}, figsize=(10, 6))

# Step 5: Plot the cluster centre values as a line on the radar chart
ax.plot(theta, first_row_centers, linewidth=2, color='blue', marker='o', label='Centres')

# Step 6: Plot a red baseline representing the origin (zero line)
ax.plot(theta, np.zeros_like(first_row_centers), color='red', linestyle='--', label='Average')

# Step 7: Label each spoke of the radar chart with the corresponding variable name
ax.set_xticks(theta)
ax.set_xticklabels(cluster_centers.columns, rotation=45, ha='right', fontsize=6)

# Optional: Add a legend and tidy layout
plt.title("Cluster 0 Profile Across Census Variables", fontsize=12, pad=20)

# Step 8: Display the plot
plt.tight_layout()
plt.show()


INSERT WHERE HIGH OR LOW COMPARED TO AVERAGE AND GROUP CLUSTER TITLE 

In [ ]:
# Step 1: Select the centre values for Cluster 1 (the first cluster)
second_row_centers = cluster_centers.iloc[1, :] 

# Step 2: Count the number of features (i.e., variables used in clustering)
num_features = len(second_row_centers)

# Step 3: Generate evenly spaced angles (in radians) for each feature around a circle
theta = np.linspace(0, 2 * np.pi, num_features, endpoint=True)

# Step 4: Create a polar plot using matplotlib
fig, ax = plt.subplots(subplot_kw={'projection': 'polar'}, figsize=(10, 6))

# Step 5: Plot the cluster centre values as a line on the radar chart
ax.plot(theta, second_row_centers, linewidth=2, color='blue', marker='o', label='Centers')

# Step 6: Plot a red baseline representing the origin (zero line)
ax.plot(theta, np.zeros_like(second_row_centers), color='red', linestyle='--', label='Average')

# Step 7: Label each spoke of the radar chart with the corresponding variable name
ax.set_xticks(theta)
ax.set_xticklabels(cluster_centers.columns, rotation=45, ha='right', fontsize=8)

# Optional: Add a legend and tidy layout
plt.title("Cluster 1 Profile Across Census Variables", fontsize=12, pad=20)

# Step 8: Display the plot
plt.tight_layout()
plt.show()

INSERT WHERE HIGH OR LOW COMPARED TO AVERAGE AND GROUP CLUSTER TITLE 

In [ ]:
# Step 1: Select the centre values for Cluster 2 (the first cluster)
third_row_centers = cluster_centers.iloc[2, :] 

# Step 2: Count the number of features (i.e., variables used in clustering)
num_features = len(third_row_centers)

# Step 3: Generate evenly spaced angles (in radians) for each feature around a circle
theta = np.linspace(0, 2 * np.pi, num_features, endpoint=True)

# Step 4: Create a polar plot using matplotlib
fig, ax = plt.subplots(subplot_kw={'projection': 'polar'}, figsize=(10, 6))

# Step 5: Plot the cluster centre values as a line on the radar chart
ax.plot(theta, third_row_centers, linewidth=2, color='blue', marker='o', label='Centers')

# Step 6: Plot a red baseline representing the origin (zero line)
ax.plot(theta, np.zeros_like(third_row_centers), color='red', linestyle='--', label='Average')

# Step 7: Label each spoke of the radar chart with the corresponding variable name
ax.set_xticks(theta)
ax.set_xticklabels(cluster_centers.columns, rotation=45, ha='right', fontsize=8)

# Optional: Add a legend and tidy layout
plt.title("Cluster 2 Profile Across Census Variables", fontsize=12, pad=20)

# Step 8: Display the plot
plt.tight_layout()
plt.show()

INSERT WHERE HIGH OR LOW COMPARED TO AVERAGE AND GROUP CLUSTER TITLE 

In [ ]:
# Step 1: Select the centre values for Cluster 3 (the first cluster)
fourth_row_centers = cluster_centers.iloc[3, :] 

# Step 2: Count the number of features (i.e., variables used in clustering)
num_features = len(fourth_row_centers)

# Step 3: Generate evenly spaced angles (in radians) for each feature around a circle
theta = np.linspace(0, 2 * np.pi, num_features, endpoint=True)

# Step 4: Create a polar plot using matplotlib
fig, ax = plt.subplots(subplot_kw={'projection': 'polar'}, figsize=(10, 6))

# Step 5: Plot the cluster centre values as a line on the radar chart
ax.plot(theta, fourth_row_centers, linewidth=2, color='blue', marker='o', label='Centers')

# Step 6: Plot a red baseline representing the origin (zero line)
ax.plot(theta, np.zeros_like(fourth_row_centers), color='red', linestyle='--', label='Average')

# Step 7: Label each spoke of the radar chart with the corresponding variable name
ax.set_xticks(theta)
ax.set_xticklabels(cluster_centers.columns, rotation=45, ha='right', fontsize=8)

# Optional: Add a legend and tidy layout
plt.title("Cluster 3 Profile Across Census Variables", fontsize=12, pad=20)

# Step 8: Display the plot
plt.tight_layout()
plt.show()

INSERT WHERE HIGH OR LOW COMPARED TO AVERAGE AND GROUP CLUSTER TITLE 

In [ ]:
# Step 1: Select the centre values for Cluster 4 (the first cluster)
fifth_row_centers = cluster_centers.iloc[4, :] 

# Step 2: Count the number of features (i.e., variables used in clustering)
num_features = len(fifth_row_centers)

# Step 3: Generate evenly spaced angles (in radians) for each feature around a circle
theta = np.linspace(0, 2 * np.pi, num_features, endpoint=True)

# Step 4: Create a polar plot using matplotlib
fig, ax = plt.subplots(subplot_kw={'projection': 'polar'}, figsize=(10, 6))

# Step 5: Plot the cluster centre values as a line on the radar chart
ax.plot(theta, fifth_row_centers, linewidth=2, color='blue', marker='o', label='Centers')

# Step 6: Plot a red baseline representing the origin (zero line)
ax.plot(theta, np.zeros_like(fifth_row_centers), color='red', linestyle='--', label='Average')

# Step 7: Label each spoke of the radar chart with the corresponding variable name
ax.set_xticks(theta)
ax.set_xticklabels(cluster_centers.columns, rotation=45, ha='right', fontsize=8)

# Optional: Add a legend and tidy layout
plt.title("Cluster 4 Profile Across Census Variables", fontsize=12, pad=20)

# Step 8: Display the plot
plt.tight_layout()
plt.show()

INSERT WHERE HIGH OR LOW COMPARED TO AVERAGE AND GROUP CLUSTER TITLE 

### Q10. Map the final cluster groups

In [ ]:
list(z_score_df.columns)

In [ ]:
z_score_df.drop([
 'pop_density',
 'age_20_24',
 'age_25_29',
 'age_30_34',
 'age_35_39',
 'eth_white',
 'male',
 'emp_active',
 'emp_unemp',
 'inactive_retired',
 'inactive_sick',
 'occ_prof',
 'occ_sales',
 'occ_elem',
 'health_very_good',
 'health_bad',
 'disab_limited_little',
 'tenure_owned',
 'tenure_rented',
 'deprived_2dim',
 'deprived_3dim'], axis=1, inplace=True)
z_score_df.info()

In [ ]:
# Concatenate the resulting tables.
final_df = pd.concat([birmingham_census_data, z_score_df], axis=1, ignore_index=False)
final_df.head()

In [ ]:
final_df.columns

In [ ]:
final_df.explore(column='Cluster', cmap='Set1', tiles='CartoDB positron')

### Q11. Finish the analysis by naming the final clusters and plotting a final map that includes the census values and the provided names.

In [ ]:
def rename_column(x): 
    x = str(x)  # Convert integer to string
    x = x.replace("0", "Title") # Replacing for the categories defined earlier. 
    x = x.replace("1", "Title")
    x = x.replace("2", "Title")
    x = x.replace("3", "Title")
    x = x.replace("4", "Title")
    return x

final_df['Cluster'] = final_df['Cluster'].apply(rename_column)

In [ ]:
final_df.explore(column='Cluster', cmap='Set1', tiles='CartoDB positron')

### Q12. Finally, acknowledge the subjective nature of classification and make analytical decisions to produce an optimum classification for your specific purpose. Reflect on the challenges and insights gained during the classification process. Ensure you document your analytical decisions and the rationale behind any important decision. Once your geodemographics are constructed, describe the potential use cases for the geodemographic classification you have built based on your initial goal.

Finally, acknowledge the subjective nature of classification and make analytical decisions to produce an optimum classification for your specific purpose. Reflect on the challenges and insights gained during the classification process. Ensure you document your analytical decisions and the rationale behind any important decision. Once your geodemographics are constructed, describe the potential use cases for the geodemographic classification you have built based on your initial goal

#### Challenges and Insights 

To start with the challenges, defining the characteristics of a cluster was difficult to assess. The word choice and the most important factors to identity cluster by seemed to be reductive in light of all the data analysed. Moreover, by creating these categories, I felt that residents who were in the minority within these OAs were not accounted for. 
Applying the categories to a more limited subset of the census data (ie a certain age range, etc.) made it difficult to determine which of the population subgroup defnitions were appropriate. Likely, ground-truthing and qualitative interviews would need to be conducted to verify the geodemographic data presented. 

Another issue is the stocastic nature of KMeans itself. This means that everytime the method is run, there is a slightly different output, which makes the data hard to reproduce and use for further analysis. I attempted to rectify this by specifying the random_state number, but more deterministic modelling techniques might be appropriate if this were to be actually used to inform a company or social service provider. 

In term of insights, there are clear spatially distributed patterns of suburban achievers on the outskirts of Birmingham. Commuters with families also made up a large marjority of the OAs, but despite this, there are clear areas that are predominantly younger students and comfortable cosmopolitans. 

#### Potential Use Cases 
In choosing the variables to make this classification, I had in mind a younger demographic, particularly for use by an advertising agency. This informed my selection of the age demographic and the availability of transportation. For example, any advertisements for a cologne or perfume brand would likely want to know the education level and employment type of the people in certain neighborhoods; this would inform the word choice used in advertising. Additionally, knowing whether people are commuters or not will inform the placement of advertisements. To advertise a cologne/perfume to the Suburban Achievers cluster, agencies might decide to invest more in large billboards by roadways. On the other hand, if the target audience are students, understanding their employment type and their spatial distrubution would make for more effective ads and inform their pricing calculations. This can aid the company or brand by providing an understanding of who their consumers are and what values they might share. 

Another potential use case for this is for public health interventions. For example, if there is a public health intervention on sexual health and STI testing, understanding the employment type, language preference/understanding, and income would be very relevant factors for targeting certain interventions and for defining the promotional material used. Interventions on public health issues would also be different for families versus single persons, and might need to reflect employment type and whether the population is a student or not. 

# Lab No 7: Spatial Microsimulation (3 Challenges)

## Challenge 1: 

## Challenge 2: 

## Challenge 3: 

# Final Remarks (limitations, barriers, and any additional comments)